# Principles of Digital Communications - Project

The project is divided in 3 milestones:
- milestone 1 is transforming a string into an array of bits, and an array of bits into a string (with utf-8 encoding)
- milestone 2 is creating a transmitter and receiver in the case of a noiseless channel
- milestone 3 is taking into account the white gaussian noise in our channel

In [1]:
import numpy as np

In [2]:
string80 = "I wish my sentence had length 80, otherwise my communication scheme would not..."
# string80 = "h"

In [3]:
len(string80)

80

In [4]:
len(bytearray(string80, 'utf-8'))

80

## Milestone 1 : 

Converts a string to an array of bits (UTF-8 encoding)

In [5]:
def string_to_bits(string):
    arr = bytearray(string, 'utf-8')
    return np.unpackbits(arr)

Converts an array of bits to a string

In [6]:
def bits_to_string(bits):
    b = bytearray(np.packbits(bits))
    return str(b, 'utf-8', errors='replace')

## Milestone 2:

Simulates the channel without noise 

In [7]:
def noiseless_channel(channel_input):
    channel_input = np.clip(channel_input,-1,1)
    erased_index = np.random.randint(3) 
    channel_input[erased_index:len(channel_input):3] = 0
    return channel_input

The transmitter :

In [8]:
def mod3_transmitter(bits):
    m = int(len(bits)/2)
    result = []
    for i in range(0,m):
        result.append(bits[i])
#         print(str(i) + " : " + str(bits[i]))
        result.append(bits[m+i])
#         print(str(m+i)+ " : " + str(bits[m+i]))
        result.append(np.bitwise_xor(bits[i],bits[m+i]))
#         print("xor : " + str(np.bitwise_xor(bits[i],bits[m+i])))
    return result

And the decoder on receiver side

In [9]:
def mod3_receiver(array):
    array = np.clip(array,0,1)
    k = int(len(array)/3)
    a1 = []
    a2 = []
    a3 = []
    zeros = []
    for i in range(0,k):
        a1.append(int(array[3*i]))
        a2.append(int(array[3*i+1]))
        a3.append(int(array[3*i+2]))
        zeros.append(0)
    if a1 == zeros:
        a1 = np.bitwise_xor(a2,a3)
    elif a2 == zeros:
        a2 = np.bitwise_xor(a1,a3)
    result = np.concatenate((a1,a2))
    return result

The whole communiation :

In [10]:
print("input string : " + string80)
input_bits = string_to_bits(string80)
# print("input in bits : " + str(input_bits))
X = mod3_transmitter(input_bits)
# print("X : " + str(X) )
Y = noiseless_channel(X)
# print("Y :" + str(Y))
output_bits = mod3_receiver(Y)
# print("output bits :" + str(output_bits))
output_string = bits_to_string(output_bits)
print("output string : " +output_string)

input string : I wish my sentence had length 80, otherwise my communication scheme would not...
output string : I wish my sentence had length 80, otherwise my communication scheme would not...


In [11]:
output_string == string80

True

## Milestone 3: 

Simulates the channel with noise

In [12]:
def bin_to_int(bits):
    a = np.flip(bits)
    result = 0
    for k in range(0,len(a)):
        result += a[k]*(2**k)
    return result

In [13]:
def int_to_bin(n, k) :
    result =[]

    for s in format(n, 'b') :
        result.append(int(s))
    return np.array(np.zeros(k - len(result), int).tolist() + result)

In [14]:
def int_to_bits(number, size):
    result = np.zeros(size, dtype = int)
    b = bin(number)
    i = len(b)-1
    j = 0
    while i > 1:
        result[j] = int(b[i])
        i-=1
        j+=1
    result = result[:size]
    return np.flip(result)

In [15]:
def channel(channel_input):
    channel_input = np.clip(channel_input,-1,1)
    erased_index = np.random.randint(3) 
    channel_input[erased_index:len(channel_input):3] = 0
    return channel_input + np.sqrt(10)*np.random.randn(len(channel_input))

### Orthogonal codes

In [16]:
#creates a codebook of size 2**k
def create_codebook(k):
    C_i0 = [[1]]
    if k == 0:
        return C_i0
    for i in range(0,k):
        C_i1 = []
        for c in C_i0:
            C_i1.append(np.concatenate((c,c)))
            C_i1.append(np.concatenate((c,np.dot(-1,c))))
        C_i0 = C_i1
    return np.array(C_i1)
        

In [17]:
def orthogonal_encoder(bits):
    k = 9
    ws = 9
    n_words = int(len(bits)/ws)
   
    codebook = create_codebook(k)
    result = []
    for i in range(0,n_words):
        u = int(ws*i)
        v = int((i+1)*ws)
        b = bits[u:v]
        index = bin_to_int(b)
        result.append(codebook[index])
    return np.array(result).flatten(), codebook

In [18]:
def orthogonal_decoder(bits, codebook):
    k = 9
    ws = 9
    word_size = len(codebook[0])
    n_words = int(len(bits)/word_size)
    result = []
    maximum = 0
    closest_word = 0
    
    for i in range(0,n_words):
        maximum = 0
        closest_word = 0
        u = int(word_size*i)
        v = int(word_size*i+word_size)
        r = [np.linalg.norm(c-bits[u:v]) for c in codebook[:word_size]]
        closest_word = np.argmin(r)
        result.append(int_to_bits(closest_word, ws))
    
    return result

### Guessing H :

In [19]:
def H_guesser(array):
    #argmin_j |y_j|^2
    a1 = array[0::3]
    a2 = array[1::3]
    a3 = array[2::3]
    y1 = (np.linalg.norm(a1))**2
    y2 = (np.linalg.norm(a2))**2
    y3 = (np.linalg.norm(a3))**2
    return np.argmin([y1,y2,y3])

In [20]:
def replace(bits, H): 
    b = bits
    for i in range(0, len(b)):
        if (i%3 == H) : 
            b[i] = 0
    return b

## Explanation of the communication scheme:
### 1. Convert the string to an array of bits


### 2. Translate the array to a list of codewords, obtained from orthogonal codes.

For k = 2, this is what the codebook would look like:

$ \mathcal{C} = \{ \begin{bmatrix}1\\1\\1\\1\end{bmatrix}, \begin{bmatrix}1\\1\\-1\\-1\end{bmatrix}, \begin{bmatrix}1\\-1\\1\\-1\end{bmatrix}, \begin{bmatrix}1\\-1\\-1\\1\end{bmatrix} \}$


At this step, we can tune parameters to obtain the best result:
- $2^k$: the codeword size (and codebook length)
- ws and n_words: the word size and number of words, which define the minimum codebook size and the amount of codewords that will be transmitted

We obtain our block length n: $n = 2^k * $nw , which has to be less than 60'000.

### 3. The array of length n goes through the communication channel.

$ X = \begin{bmatrix} 1 & 1 & 1 & -1 & ... & -1 & -1 & 1\end{bmatrix}$ $\rightarrow$ _channel_ $\rightarrow$ $ Y = \begin{bmatrix} 3.43 & 2.24 & -7.32 & -3.2 & ... & -0.54 & 1.32 & 6.32\end{bmatrix}$


### 4. Guess H using a MAP estimator, set all _deleted_ elements to 0.

$ H = argmin_{i=1,2,3} ||Y_i||$ 

with $Y_i$'s being the 3 subarrays of $Y$, obtained by taking alternatively 1/3 elements.


### 6. Use a MAP estimator to estimate each codeword. Translate them back to an array of bits.

$ \hat{x_l} = argmin_{j=1,...,2^k} ||c_j - y_l||$ for $y_l$ words of $Y$ and $c_j \in \mathcal{C}$


### 7. Finally, convert the array back to string, with utf-8 encoding. Then compare the string with the original one.

### The communication :

In [21]:
string80 = "As has been pointed out several times, there is no such thing as a random number"
len(string80)

80

In [42]:
def comm(string=string80 ):
    input_bits = string_to_bits(string80)
    A, codebook = orthogonal_encoder(input_bits)
    Y = channel(A)
    H = H_guesser(Y)
    Y_1 = replace(Y, H)
    output_bits = orthogonal_decoder(Y_1, codebook)
    out = bits_to_string(output_bits)
    return out

In [43]:
comm()

'I wish my sentence had length 80, otherwise my communication scheme would not...'

In [49]:
def error_prob():
    yes = 0
    total = 1000
    for i in range(0,total):
        if comm() == string80:
            yes+=1
    return 1 - (yes/total)

In [79]:
error_prob()

0.32299999999999995